In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_path_after, batch_size=8, color_mode='grayscale', image_size=(256, 256))
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=8, figsize=(24, 6))
for i, img in enumerate(batch[0][:8]):
    class_name = labels[batch[1][i]]
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(class_name)
    axes[i].set_axis_off()

plt.tight_layout()
plt.show()

**_deprecated_**
<br>
The process below is called _color quantization_.
<br>
For this task I used Kmeans algorithm to cluster color values in each image.
<br>
So instead of dealing with 255 shades of grey we have only 5 colors per image. 

In [ ]:
from sklearn.cluster import KMeans
import cv2 as cv

all_files = lambda label: os.listdir(os.path.join(data_path, label))

_ = 0
for label in labels:
    break
    for i, file in enumerate(all_files(label)):
        
        # if _ >= 10:
        #     break
            
        file_path = os.path.join(data_path, label, file)

        img = cv.imread(file_path, cv.IMREAD_GRAYSCALE)

        width, height = img.shape[0], img.shape[1]

        reshaped_image = np.reshape(img, (width * height, 1))

        kmeans = KMeans(n_clusters=5, random_state=0)
        target = kmeans.fit_predict(reshaped_image)
        color_space = kmeans.cluster_centers_
        
        img_after = np.reshape(color_space[target], img.shape)
        
        cv.imwrite(os.path.join(data_path_after, label, f"{label}_{i}.png"), img_after)

print('Done!')

In [ ]:
import splitfolders

input_folder = "D:\\data-science\\brain-tumor-data\\processed_images"
output_folder = "D:\\data-science\\brain-tumor-data\\processed_images_split"

splitfolders.ratio(input=input_folder, output=output_folder, ratio=(.7, .2, .1))

In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import numpy as np
import os

data_path = 'D:\\data-science\\brain-tumor-data\\images'
data_path_split = data_path + '_split'

test = tf.keras.utils.image_dataset_from_directory(os.path.join(data_path_split, 'test'), batch_size=8, color_mode='grayscale', image_size=(256, 256))
label_names = test.class_names
# val = tf.keras.utils.image_dataset_from_directory(os.path.join(data_path_split, 'val'), batch_size=8, color_mode='grayscale', image_size=(256, 256))
test = test.map(lambda x, y: (x/255, y))
model = tf.keras.models.load_model("tumor-type-cnn-v1.h5")

Found 308 files belonging to 3 classes.


In [2]:
# Recompile the model with metrics
# model.compile(optimizer=Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

# Now you can evaluate or train the model
# model.evaluate(val)

In [3]:
y_true = []
y_pred = []

for imgs, labels in test:
    batch_predictions = model.predict(imgs, verbose=0)
    
    y_true.extend(labels)
    y_pred.extend(batch_predictions)

# y_true = np.concatenate(y_true, axis=0)
# y_pred = np.concatenate(y_pred, axis=0)
y_pred = np.argmax(y_pred, axis=1)

In [4]:
from sklearn.metrics import confusion_matrix, classification_report

report = classification_report(y_true, y_pred, target_names=label_names)
print(report)

              precision    recall  f1-score   support

      glioma       0.97      0.98      0.97       143
  meningioma       0.95      0.86      0.91        72
   pituitary       0.94      0.99      0.96        93

    accuracy                           0.95       308
   macro avg       0.95      0.94      0.95       308
weighted avg       0.95      0.95      0.95       308

